<a href="https://colab.research.google.com/github/nikitamaia/tensorflow-examples/blob/main/task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FROM PROTOTYPE TO PRODUCTION
#### Train an image classification model on the [flower photos dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers). 

## 🌹 🌸 🌺 🌼 🌻

This notebook is part of the From Prototype to Production video series, which covers the basics of training, scaling, and deploying custom models on Google Cloud.

Import the necessary libraries

In [ ]:
import tensorflow as tf
import os
import pathlib

Define variables

In [ ]:
NUM_CLASSES = 5
EPOCHS=10
BATCH_SIZE = 32

IMG_HEIGHT = 180
IMG_WIDTH = 180

Download the data

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

228827136/228813984 [==============================] - 10s 0us/step


Verify that images have been downloaded by printing the image count.

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3670


Define `create_datasets` function, which uses the [`tf.keras.utils.image_dataset_from_directory` utility](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) to load the images off disk into a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [ ]:
def create_datasets(data_dir, batch_size):
  train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size)
  
  validation_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=batch_size)

  train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
  validation_dataset = validation_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

  return train_dataset, validation_dataset

Define `create_model` function which creates our image classification model using the [`tf.keras` Sequential API](https://www.tensorflow.org/guide/keras/sequential_model).

In [ ]:
def create_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMG_HEIGHT, IMG_WIDTH),
    tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
  ])
  return model

Create the training and validation datasets

In [ ]:
train_dataset, validation_dataset = create_datasets(data_dir, BATCH_SIZE)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


Create and compile the model

In [ ]:
model = create_model()
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

Fit the model

In [ ]:
history = model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=EPOCHS
)

Epoch 1/10
92/92 [==============================] - 103s 1s/step - loss: 1.3148 - accuracy: 0.4407 - val_loss: 1.0995 - val_accuracy: 0.5518
Epoch 2/10
92/92 [==============================] - 95s 1s/step - loss: 0.9906 - accuracy: 0.6196 - val_loss: 0.9941 - val_accuracy: 0.6240
Epoch 3/10
92/92 [==============================] - 96s 1s/step - loss: 0.8238 - accuracy: 0.6866 - val_loss: 0.9361 - val_accuracy: 0.6458
Epoch 4/10
92/92 [==============================] - 95s 1s/step - loss: 0.6113 - accuracy: 0.7800 - val_loss: 0.9760 - val_accuracy: 0.6417
Epoch 5/10
92/92 [==============================] - 96s 1s/step - loss: 0.3852 - accuracy: 0.8651 - val_loss: 1.0474 - val_accuracy: 0.6417
Epoch 6/10
92/92 [==============================] - 95s 1s/step - loss: 0.2125 - accuracy: 0.9305 - val_loss: 1.2383 - val_accuracy: 0.6294
Epoch 7/10
92/92 [==============================] - 97s 1s/step - loss: 0.1334 - accuracy: 0.9561 - val_loss: 1.4411 - val_accuracy: 0.6267
Epoch 8/10
92/92 [=

Save the model

In [ ]:
model.save('model-output')

INFO:tensorflow:Assets written to: model-output/assets
